In [1]:
# Imports
import os
import numpy as np
import pandas as pd 
import csv
import matplotlib.pyplot as plt
from math import sqrt

from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data(data_name):
    '''
        Permet d'obtenir un visuel rapide et clair des données. Retourne un dataframe
        contenant les données.
        
        :params:
            data_name: nom du fichier où se trouvent les données
        
        :type params:
            data_name: string
        
        :return: dataframe contenant les données
        :rtype: pandas.dataframe
    '''
    _, ext = os.path.splitext(data_name)
    
    if ext == '.csv' : 
        with open(data_name, newline='') as csvfile:
            dialect = csv.Sniffer().sniff(csvfile.readline(), [',',';'])
            csvfile.close()
        df = pd.read_csv(data_name, dialect=dialect)
    
    if ext == '.json' :
        print('json')
        
    print(descript_df(df))
    return df 

In [3]:
import importlib

def load_all_data(path):
    '''
        Ouvre et lis le fichier passer en parmètre, reconnais seulement les types de
        fichier supportés par pandas : CSV, JSON, HTML, Local clipboard, MS Excel,
        HDF5 Format, feather Format, Parquet Format, Msgpack, Stata, SAS, Python Pickle
        Format, SQL, Google Big Query. Retourne les données lu.
        
        :params:
            path: path of the file
        
        :type params:
            path: string
        
        :return: object containing the data loaded in memory, or return -1 if type
                 not recognize.
    '''
    #On récupère le nom de l'extension du fichier
    type = path.split(".")[-1]
    #Selection de la bonne fonction de pandas à utiliser
    func_to_call = 'read_{}'.format(type)
    #Récupération de l'attribut de la fonction pour l'appeler
    try :
        func = getattr(pd, func_to_call)
    except :
        print("Pas de fonction disponible dans pandas pour lire les données")
        return -1
    #Lecture des données
    try :
        return func(path)
    except :
        print("path incorrect")
        return -1

In [4]:
def descript_df(dataframe):
    '''
        Permet de faire une description rapide du dataframe de sortie. Retourne une
        description rapide des données.
        
        :params:
            dataframe: de faire une description rapide du dataframe de sortie
        
        :type params:
            dataframe: pandas.dataframe
        
        :return: description rapide des données
        :rtype: string
    '''   
    print('Matrice de corrélation : \n')
    corr = dataframe.corr()
    corr_color = plt.matshow(corr, cmap=plt.cm.Reds)
    
    # Pour chaque colonne, montrer la répartition des valeurs (vérifier les valeurs aberrantes)
    # Kde et Histogramme
    i = 1
    for column in dataframe:
        i += 1
        plt.figure(i, figsize=(15,3))
        plt.subplot(121)
        dataframe[column].plot.kde()
        plt.title('Répartition de ' + column + ' : ')
        plt.subplot(122)
        dataframe[column].hist()
        plt.title('Histogramme de ' + column + ' : ')
    
    display = corr
    return display

In [5]:
# Fonction permettant de créer les bases d'apprentissage et de test
# params :
#    dataframe : dataframe contenant les données
#    column_target : string de la colonne-cible à estimer
#    test_size : ratio de la base de test (30% par défaut)
# returns :
#    base : tuple des bases d'apprentissage et de test
def split_base(dataframe, column_target, test_size = 0.3):
    
    # Gestion erreur test_size
    if(test_size <= 0 or test_size >= 1):
        return "Erreur sur le ratio de test_size : vérifier que test_size est compris entre 0 et 1. \n"
    
    # Gestion erreur column_target comme string
    elif (isinstance(column_target, str) == False):
        return "Erreur sur le type de column_target : vérifier que column_target est bien une string. \n"
    
    else:
        train, test = train_test_split(dataframe, test_size = test_size)

        X_train = train.drop(column_target, axis=1)
        y_train = train[column_target]

        X_test = test.drop(column_target, axis=1)
        y_test = test[column_target]
        
        base = X_train, y_train, X_test, y_test
        
        return base

In [18]:
def perform_on_pipeline(pipeline, bool_type_modele, base):
    '''
        Permet d'appeler les bons indicateurs de performances et de récupérer
        les informations utiles selon la pipeline passé en paramètre. Retourne les
        performances associées.
        
        :params:
            pipeline : sortie de la pipeline
            bool_type_modele : booléen pour le type du modèle (0: régression / 1: classification)
            base : tuple des bases d'apprentissage et de test
        
        :type params:
            bool_type_modele: boolean
            base: tuple
        
        :return: les performances des différents indicateurs et graphiques
    '''
    if (bool_type_modele == 0) : 
        print("Choix du type d'estimateur : Régression \n")
        perfs = indic_perform_reg(pipeline, base)
        
        # poster les scores dans la database 
    
    else:
        print("Choix du type d'estimateur : Classification \n")
        perfs = indic_perform_class(pipeline, base)
        
        # poster les scores dans la database
        
    return perfs
    

In [7]:
# Fonction permettant d'obtenir les résultats des indicateurs de performances d'une régression
# params :
#    pipeline : sortie de la pipeline
#    base : tuple des bases d'apprentissage et de test
# returns :
#    rmse : score RMSE
#    r2 : score R²
#    cross_val : score de cross-validation
def indic_perform_reg(pipeline, base):
    ''' # Fonction permettant d'obtenir les résultats des indicateurs de performances d'une régression
        # params :
        #    pipeline : sortie de la pipeline
        #    base : tuple des bases d'apprentissage et de test
    '''
    # Pré-process ?
    pipeline_fit = pipeline.fit(base[0],base[1]) # besoin du fit ou seulement du préd ? 
    pred = pipeline_fit.predict(base[2]) # besoin du predict ou déjà fait ? 
    
    # Calcul du RMSE
    rmse = sqrt(mean_squared_error(base[3], pred))
    
    # Calcul du R²
    r2 = r2_score(base[3], pred)
    
    # Cross-Validation
    cross_val = np.mean(cross_val_score(pipeline, base[2], base[3], cv=5))
    
    return rmse, r2, cross_val

In [8]:
# Fonction permettant d'obtenir les résultats des indicateurs de performances d'une classification
# params :
#    pipeline : sortie de la pipeline
#    X_test : variables de test pour évaluer les performances du modèle
#    y_test : valeurs de test pour évaluer les performances du modèle
# returns :
#    matrice_confu : matrice de confusion
#    classif_report : classification report
def indic_perform_class(pipeline, X_test, y_test):
    # Pré-process ?
    pipeline_fit = pipeline.fit(X_train,y_train) # besoin du fit ou seulement du préd ? 
    pred = pipeline_fit.predict(X_test) # besoin du predict ou déjà fait ? 
    
    # Calcul Matrice de confusion
    matrice_confu = False
    
    # Calcul du Classification Report
    classif_report = False
    
    return matrice_confu, classif_report

# Test

In [16]:
# Loading du dataframe et description
df_test = load_all_data("./../data/headbrain.csv")
#descript_df(df_test)

In [13]:
# Nettoyage en amont / Choix des colonnes à utiliser ???

# Splitting entre apprentissage et test
base = split_base(df_test, "Brain Weight(grams)")

# Pipeline à utiliser
pipe_test = Pipeline([
    ('features', StandardScaler()),
    ('estimator', neighbors.KNeighborsRegressor())   
])

# Calculer les performances selon le type d'estimateur
perform = perform_on_pipeline(pipe_test, 0, base)
perform

Choix du type d'estimateur : Régression 



(78.62805196337727, 0.566006456290973, 0.472519474482436)

In [19]:
help(perform_on_pipeline)

Help on function perform_on_pipeline in module __main__:

perform_on_pipeline(pipeline, bool_type_modele, base)
    Permet d'appeler les bons indicateurs de performances et de récupérer
    les informations utiles selon la pipeline passé en paramètre. Retourne les
    performances associées.
    
    :param a: sortie de la pipeline
    :param b: booléen pour le type du modèle (0: régression / 1: classification)
    :param c: tuple des bases d'apprentissage et de test
    
    :type b: boolean
    :type c: tuple
    
    :return: les performances des différents indicateurs et graphiques

